<a href="https://colab.research.google.com/github/drambaut/NER-securtiy-project/blob/main/CyNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* https://github.com/aiforsec/CyNER/blob/main/dataset/mitre/train.txt
* https://arxiv.org/pdf/2204.05754v1.pdf
* https://ebiquity.umbc.edu/_file_directory_/papers/1152.pdf

In [ ]:
!pip install git+https://github.com/aiforsec/CyNER.git

In [ ]:
import cyner

In [ ]:
cfg = {'checkpoint_dir': '.ckpt',
        'dataset': 'mitre',
        'transformers_model': 'xlm-roberta-large',
        'lr': 5e-6,
        'epochs': 5,
        'max_seq_length': 128}
model = cyner.TransformersNER(cfg)
model.train()

# NB. `model` cannot be used for prediction; reload:

model4 = cyner.CyNER(transformer_model='.ckpt', use_heuristic=False, flair_model=None)

In [ ]:
#model = cyner.CyNER(transformer_model='xlm-roberta-large', use_heuristic=False, flair_model=None)

model2 = cyner.CyNER(transformer_model='xlm-roberta-large', use_heuristic=True, flair_model=None, priority='HTFS')

text = 'Advanced SystemCare Pro 15.3 2022 Lifetime Access How to protect my PC from viruses? Fortunately as an all-in-one antivirus utility Advanced SystemCare Ultimate provides you the best and easy method to protect your Windows simply in real-time. It does not just block all potential threats from viruses spyware ransomware and other types of malware with the 23% larger database. Also offers a double-safe online experience along with the latest Bitdefender engine. This all-in-one PC utility is always the best choice to protect your security of privacy and optimize the computer performance with one click. Browser Protection Clear each track even on the latest Microsoft Edge (Chromium) for a securer and smoother Internet browsing experience. NEW Real-time Protection Prevent from viruses spyware ransomware trojans and more potential threats in real-time. HOT Privacy Shield Block any unauthorized and secret access. Sweep each online activity traces. HOT Antivirus Database Ensure up-to-date Windows protection to scan find and remove tons of viruses in one click. IMPROVED Startup Optimization Manage overuse startup items and stop them easily for a faster system response speed. IMPROVED Junk Files Clean One-click to scan and clean invalid entries and junk files to reduce disk space and the risk of Windows crashes. SAFE Secure Your Private Data Your computer is a big house full of personal information. How to keep your private data secure? What Advanced SystemCare Ultimate does is to create a security shield to prevent such sensitive personal data such as files photos contacts and digital fingerprints from being stolen by unauthorized individuals. And stop hackers from taking over your PC remotely. If there is unauthorized access to your private data the program will block it upon detection. PRIVATELY Surf the Internet without Risks Before heading to a new website you may probably ask yourself “Is it safe to open this website?”. Advanced SystemCare Ultimate always keeps you safe and private online. The enhanced browser protection integrates email protection ads removal surfing protection anti-tracking and homepage advisor providing a secure surfing environment. It also provides one-stop protection against phishing scams invasive online tracking and other browser-based threats which also support surfing on the latest Microsoft Edge (Chromium). SMOOTH Speed up Your Windows Advanced SystemCare Ultimate is also a system optimizer to speed up your slow PC. It cleans up tons of junk files temporary files and caches in Dropbox Steam Chrome and other browsers and software for a bigger space of hard disk and faster system response. Furthermore the 42% bigger database of Startup Optimizer can easily manage excessive startup items and services to save boot time. At the same time the enhanced Internet Booster accelerates Internet speed for a smooth PC experience. Refund policy I do not replace or refund account with 2fa. you are responsible for any security issue that come up while working with account. if you request assistance upon purchase i will try my best to assist you. For none account product contact me for help if you run into difficulties'
#text = 'Proofpoint report mentions that the German-language messages were turned off once the UK messages were established, indicating a conscious effort to spread FluBot 446833e3f8b04d4c3c2d2288e456328266524e396adbfeba3769d00727481e80 in Android phones.'

entities = model2.get_entities(text)

for e in entities:
    print(e)

# NER model spaCy

In [ ]:
import spacy

In [ ]:
nlp=spacy.load('en_core_web_sm')
ner=nlp.get_pipe("ner")
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [ ]:
with open('train.txt', encoding='utf-8') as f:
  lines = f.readlines()

In [ ]:
TRAIN_DATA = []

text = ""
dicc_entities = {"entities": []}
for l in lines:
  e = l.replace('\t', ' ')
  e = e.replace('\n', '')
  a = e.split(' ')
  if len(a) == 1:
    TRAIN_DATA.append((text,dicc_entities))
    text = ""
    dicc_entities = {"entities": []}
  else:
    last_len = len(text)
    text += a[0] + " "
    if a[1] != 'O':
      aux_tuple = (last_len, len(text)-2, a[1])
      dicc_entities["entities"].append(aux_tuple)

  
print(TRAIN_DATA)

[('Super Mario Run Malware # 2 – DroidJack RAT Gamers love Mario and Pokemon , but so do malware authors . ', {'entities': [(0, 4, 'B-Malware'), (6, 10, 'I-Malware'), (12, 14, 'I-Malware'), (16, 22, 'I-Malware'), (30, 38, 'B-Malware'), (40, 42, 'I-Malware'), (56, 60, 'B-System'), (66, 72, 'B-System')]}), ('A few days back , we wrote about an Android Marcher trojan variant posing as the Super Mario Run game for Android . ', {'entities': [(36, 42, 'B-System'), (44, 50, 'B-Malware'), (81, 85, 'B-System'), (87, 91, 'I-System'), (93, 95, 'I-System'), (106, 112, 'B-System')]}), ('We have found another instance of malware posing as the Super Mario Run Android app , and this time it has taken the form of DroidJack RAT ( remote access trojan ) . ', {'entities': [(56, 60, 'B-System'), (62, 66, 'I-System'), (68, 70, 'I-System'), (72, 78, 'B-System'), (125, 133, 'B-Malware'), (135, 137, 'I-Malware')]}), ('Proofpoint wrote about the DroidJack RAT side-loaded with the Pokemon GO app back in July 201

In [ ]:
def train_spacy(data,iterations):
  TRAIN_DATA = data
  nlp = spacy.blank('en')  # create blank Language class
  # create the built-in pipeline components and add them to the pipeline
  # nlp.create_pipe works for built-ins that are registered with spaCy
  if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
      

  # add labels
  for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
      ner.add_label(ent[2])

  # get names of other pipes to disable them during training
  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
  with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(iterations):
      print("Statring iteration " + str(itn))
      random.shuffle(TRAIN_DATA)
      losses = {}
      for text, annotations in TRAIN_DATA:
        nlp.update(
            [text],  # batch of texts
            [annotations],  # batch of annotations
            drop=0.2,  # dropout - make it harder to memorise data
            sgd=optimizer,  # callable to update weights
            losses=losses)
        print(losses)
  return nlp


In [ ]:
prdnlp = train_spacy(TRAIN_DATA, 20)

modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)

test_text = input("Enter your testing text: ")
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [ ]:
from spacy.util import filter_spans

for training_example  in tqdm(TRAIN_DATA): 
    text = training_example[0]
    labels = training_example[1]['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

  5%|▍         | 132/2810 [00:00<00:04, 663.48it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

 10%|█         | 287/2810 [00:00<00:03, 662.42it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skippin

 13%|█▎        | 354/2810 [00:00<00:04, 514.77it/s]

...


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skippin

 90%|████████▉ | 2527/2810 [00:03<00:00, 916.18it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

100%|██████████| 2810/2810 [00:03<00:00, 713.46it/s] 

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-27 21:11:32,173] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-27 21:11:32,182] [INFO] Pipeline: []
INFO:spacy:Pipeline: []
[2022-12-27 21:11:32,185] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-27 21:11:32,186] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-12-27 21:11:32,186] [INFO] Initialized pipeline components: []
INFO:spacy:Initialized pipeline components: []
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: []
ℹ Initial learn rate: 0.001
E    #       SCORE 
---  ------  ------
  0       0    0.00
  0     200    0.00
  0     400    0.00
  1     600    0.00
  1     800    0.00
  2    1000    0.00
  3    1200    0.00
  4    1400    0.00
  5    1600    0.00
✔ Saved pipeline to output 

In [ ]:
nlp_ner = spacy.load("model-best")
doc = nlp_ner('Advanced SystemCare Pro 15.3 2022 Lifetime Access How to protect my PC from viruses? Fortunately as an all-in-one antivirus utility Advanced SystemCare Ultimate provides you the best and easy method to protect your Windows simply in real-time. It does not just block all potential threats from viruses spyware ransomware and other types of malware with the 23% larger database. Also offers a double-safe online experience along with the latest Bitdefender engine. This all-in-one PC utility is always the best choice to protect your security of privacy and optimize the computer performance with one click. Browser Protection Clear each track even on the latest Microsoft Edge (Chromium) for a securer and smoother Internet browsing experience. NEW Real-time Protection Prevent from viruses spyware ransomware trojans and more potential threats in real-time. HOT Privacy Shield Block any unauthorized and secret access. Sweep each online activity traces. HOT Antivirus Database Ensure up-to-date Windows protection to scan find and remove tons of viruses in one click. IMPROVED Startup Optimization Manage overuse startup items and stop them easily for a faster system response speed. IMPROVED Junk Files Clean One-click to scan and clean invalid entries and junk files to reduce disk space and the risk of Windows crashes. SAFE Secure Your Private Data Your computer is a big house full of personal information. How to keep your private data secure? What Advanced SystemCare Ultimate does is to create a security shield to prevent such sensitive personal data such as files photos contacts and digital fingerprints from being stolen by unauthorized individuals. And stop hackers from taking over your PC remotely. If there is unauthorized access to your private data the program will block it upon detection. PRIVATELY Surf the Internet without Risks Before heading to a new website you may probably ask yourself “Is it safe to open this website?”. Advanced SystemCare Ultimate always keeps you safe and private online. The enhanced browser protection integrates email protection ads removal surfing protection anti-tracking and homepage advisor providing a secure surfing environment. It also provides one-stop protection against phishing scams invasive online tracking and other browser-based threats which also support surfing on the latest Microsoft Edge (Chromium). SMOOTH Speed up Your Windows Advanced SystemCare Ultimate is also a system optimizer to speed up your slow PC. It cleans up tons of junk files temporary files and caches in Dropbox Steam Chrome and other browsers and software for a bigger space of hard disk and faster system response. Furthermore the 42% bigger database of Startup Optimizer can easily manage excessive startup items and services to save boot time. At the same time the enhanced Internet Booster accelerates Internet speed for a smooth PC experience. Refund policy I do not replace or refund account with 2fa. you are responsible for any security issue that come up while working with account. if you request assistance upon purchase i will try my best to assist you. For none account product contact me for help if you run into difficulties')
colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

/usr/local/lib/python3.8/dist-packages/spacy/displacy/__init__.py:206: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
